## Importing libraries

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from collections import Counter
import matplotlib.pyplot as plt
import nltk
import re
import seaborn as sns

## Data loading and inspection

In [ ]:
ticket_df = pd.read_csv("customer_support_tickets.csv")
ticket_df.info()

In [ ]:
ticket_df.describe()

In [ ]:
ticket_df.head()

In [ ]:
ticket_df.isnull().sum()

## Data cleaning and preparation

In [ ]:
ticket_df.columns.tolist()

In [ ]:
ticket_df.rename(columns={'Ticket ID': 'ticket_id', 
                   'Customer Name': 'cust_name', 
                   'Customer Email': 'email',
                   'Customer Age': 'cust_age',
                   'Customer Gender' : 'cust_gender',
                   'Product Purchased' : 'product_purchased',
                   'Date of Purchase': 'date_of_purchase',
                   'Ticket Type' : 'ticket_type',
                   'Ticket Subject' : 'ticket_subject',
                   'Ticket Description' : 'ticket_description',
                   'Ticket Status' : 'ticket_status',
                   'Resolution': 'resolution',
                   'Ticket Priority' : 'priority',
                   'Ticket Channel' : 'channel',
                   'First Response Time' : 'first_response_time',
                   'Time to Resolution' : 'time_to_resolution',
                   'Customer Satisfaction Rating' : 'satisfaction_rate'
                  }, inplace=True)

In [ ]:
ticket_df.columns.tolist()

In [ ]:
# Select specific columns from ticket_df
key_columns = [
    'ticket_id',
    'ticket_type',
    'ticket_subject',
    'ticket_description',
    'ticket_status',
    'resolution',
    'priority',
    'channel',
    'first_response_time',
    'time_to_resolution',
    'satisfaction_rate'
 ]



# Create a new DataFrame with only the key columns
ticket_summary_df = ticket_df[key_columns].copy()

In [ ]:
ticket_summary_df.isnull().sum()

In [ ]:
ticket_summary_df[['ticket_status', 'resolution', 'first_response_time', 'time_to_resolution', 'satisfaction_rate']]

- resoution is null indicates that ticket still open
- Everytime resolutions is null time_to_resolution is null because this column indicates when the ticket is resolved
- Satisfaction is null because customer hasnt responded yet


In [ ]:
ticket_per_channel  = ticket_summary_df.groupby(['channel', 'ticket_status'])['ticket_id'].count()
ticket_per_channel

In [ ]:
# Reset index so 'channel' and 'ticket_status' become columns
ticket_plot = ticket_per_channel.reset_index()

# Set plot style
sns.set(style="whitegrid")

# Create grouped bar chart
plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=ticket_plot,
    x='channel',
    y='ticket_id',
    hue='ticket_status',
    palette='Set2'
)

# Add count labels on top of each bar
for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', fontsize=9)

# Title and labels
plt.title('Channel Tickets by Status', fontsize=14)
plt.xlabel('Channel')
plt.ylabel('Number of Tickets')
plt.legend(title='Ticket Status', loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
# Parsing dates

ticket_summary_df['first_response_time'] = pd.to_datetime(ticket_summary_df['first_response_time'], errors='coerce')
ticket_summary_df['time_to_resolution'] = pd.to_datetime(ticket_summary_df['time_to_resolution'], errors='coerce')
# Columns for null relationship

ticket_summary_df['not_checked_ticket'] = pd.isnull(ticket_summary_df['first_response_time'])
ticket_summary_df['not_resolved_ticket'] = pd.isnull(ticket_summary_df['time_to_resolution'])

# Filters

not_checked_tickets = ticket_summary_df[ticket_summary_df['not_checked_ticket']]
not_resolved_tickets = ticket_summary_df[ticket_summary_df['not_resolved_ticket']]
resolved_tickets = ticket_summary_df[~ticket_summary_df['not_resolved_ticket']] # Closed tickets

In [ ]:
not_resolved_tickets

In [ ]:
satisfaction_count = resolved_tickets.groupby('satisfaction_rate')['ticket_id'].count()

# Bar plot
satisfaction_count.plot(kind='bar')
plt.title('Satisfaction Rate on Resolved Ticket')
plt.ylabel('Number of Tickets')
plt.xlabel('Satisfaction Rate')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Re-download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Combine subject and description for analysis
text_data = ticket_summary_df['ticket_subject'].fillna('') + ' ' + ticket_summary_df['ticket_description'].fillna('')

# Tokenize and preprocess
tokens = word_tokenize(' '.join(text_data).lower())
tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation/numbers

# Remove stopwords
stop_words = set(stopwords.words('english'))
filtered_tokens = [word for word in tokens if word not in stop_words]

# Get most common keywords
word_freq = Counter(filtered_tokens)
common_words = word_freq.most_common(10)
top_keywords_df = pd.DataFrame(common_words, columns=['Keyword', 'Frequency'])




In [ ]:

# Unpack keywords and their frequencies
keywords, frequencies = zip(*common_words)

# Plot configuration
plt.figure(figsize=(12, 6))
plt.barh(keywords[::-1], frequencies[::-1], color='steelblue')  # Reverse for top-down order
plt.xlabel("Frequency")
plt.title("Reported Keywords")
plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
ticket_summary_df['response_date'] = ticket_summary_df['first_response_time'] .dt.date
ticket_summary_df['response_time(Hours)'] = ticket_summary_df['first_response_time'].dt.hour

ticket_summary_df['resolution_date'] = ticket_summary_df['time_to_resolution'] .dt.date
ticket_summary_df['resolution_time(Hours)'] = ticket_summary_df['time_to_resolution'].dt.hour
ticket_summary_df

In [ ]:
# 1. Ticket volume by type
ticket_type_counts = ticket_summary_df.groupby('ticket_type')['ticket_id'].count()

# 2. Average resolution time by ticket type
avg_resolution_time = ticket_summary_df.groupby('ticket_type')['response_time(Hours)'].mean().sort_values(ascending=False).head(10)

fig, axs = plt.subplots(3, 1, figsize=(12, 18))

# Plot 1: Ticket volume by type
sns.barplot(x=ticket_type_counts.values, y=ticket_type_counts.index, ax=axs[0], palette="Blues_d", hue=ticket_type_counts, legend=False)
axs[0].set_title("Top 10 Ticket Types by Volume")
axs[0].set_xlabel("Number of Tickets")
axs[0].set_ylabel("Ticket Type")

# Plot 2: Average resolution time
sns.barplot(x=avg_resolution_time.values, y=avg_resolution_time.index, ax=axs[1], palette="Reds_r", hue=avg_resolution_time, legend=False)
axs[1].set_title("Average Time to Resolution by Ticket Type")
axs[1].set_xlabel("Time to Resolution (Hours)")
axs[1].set_ylabel("Ticket Type")

# Plot 3: Top keywords
sns.barplot(x='Frequency', y='Keyword', data=top_keywords_df, ax=axs[2], palette="Greens_r",  hue='Keyword',legend=False)
axs[2].set_title("Top 10 Keywords in Ticket Descriptions")
axs[2].set_xlabel("Frequency")
axs[2].set_ylabel("Keyword")

plt.tight_layout()
plt.show()

Based on the customer support ticket data analysis specifically the ticket volumes, average resolution times, and common issue keywords—here are several actionable recommendations to improve response time and support efficiency:

## Prioritize High-Volume Ticket Types

🔍 Insight:
- Some ticket types consistently appear more frequently than others e.g., "Login Issues", "Billing Queries"

✅ Action:
- Create canned responses and automated workflows for repetitive ticket types.
- Build a self-service knowledge base for these high-volume issues.

## Optimize for Slowest-Resolved Categories

🔍 Insight:
- Certain ticket categories e.g."Technical Errors", "System Downtime" have the highest average resolution time.

✅ Action:
- Assign such tickets to specialized technical teams.
- Implement escalation rules for tickets older than a threshold e.g., 48 hours.
- Introduce ticket aging dashboards for real-time monitoring.

## Leverage Keyword Insights for Routing

🔍 Insight:
- Keyword frequency analysis (e.g., frequent words like "error", "login", "access", "payment") reveals common user frustrations.

✅ Action:
- Use natural language processing (NLP) to classify and auto-route incoming tickets based on keyword patterns.
- Train chatbot models on these keywords to handle tier-1 issues.

## Improve Agent Training Based on Common Issues

🔍 Insight:
- Common complaint categories indicate training gaps.

✅ Action:
- Develop scenario based training modules around the top 5 recurring problems.
- Share monthly trend reports with support staff.

## Automate First-Level Responses

🔍 Insight:
- Many tickets take long to receive even their first response, not just resolution.

✅ Action:
- Set up automated acknowledgments and intelligent chatbots to capture issue context before human intervention.
- Enable agents to see similar historical tickets as reference.

## Monitor KPIs and Continuous Feedback

✅ Action:
- Track KPIs:

    - Avg. Resolution Time

    - First Response Time

- Conduct monthly retrospectives to identify improvement areas.